In [1]:
import random
import threading

from sko.PSO import PSO
from math import exp

import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [2]:
def get_fractional_series(X0, zeta):
    """
    计算序列的分数阶
    :param X0: 待计算分数阶的序列
    :param zeta: zeta
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr


def func(XX):
    zeta, tau, gamma = XX

    # 第一列是年份，故先舍弃
    X0 = data.iloc[:train_n, 1:].to_numpy()
    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    y1 = X1[:, 0] ** (1 - gamma)

    Zy = (y1[1:] + y1[:-1]) / 2
    M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
    M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

    # 构建 B
    TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
    ONE = np.ones(train_n - 1)
    B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

    # 构建 Y
    Y = np.diff(y1)

    # 最小二乘法
    # 最小二乘法
    try:
        a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
    except:
        return 1
    #L1 = 0.00001 * (a ** 2 + b2 ** 2 + b3 ** 2 + h1 ** 2 + h2** 2)
    L2 = 0.00001 * (a ** 2 + b2 ** 2 + b3 ** 2 )

    # 求 mu
    mu1 = (1 - gamma) / (1 + 0.5 * a * (1 - gamma))
    mu2 = (1 - 0.5 * a * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))
    mu3 = (h1 * (1 - gamma) * (exp(tau) - 1)) / (1 + 0.5 * a * (1 - gamma)) / tau
    mu4 = (h2 * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))

    # 这里重新获取下原数据，以实现所有数据的预测
    # 第一列是年份，故先舍弃
    X0 = data.iloc[:, 1:].to_numpy()
    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    # 求 hat y1(1为上标的那个haty^1)
    hat_y1 = [y1[0]]
    a = []
    for m in range(2, n + 1):
        hat_ym = 0
        hat_ym += mu2 ** (m - 1) * y1[0]
        for v in range(1, m):
            a.append(m - v)
            hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
        for w in range(0, m - 1):
            hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
        hat_y1.append(hat_ym)

    hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))

    hat_x0 = []
    for k in range(2, n + 1):
        hat_x0_m = 0
        for j in range(0, k):
            hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
        hat_x0.append(hat_x0_m)

    original_x1 = data.iloc[1:, 1]
    APE = np.abs(hat_x0 - original_x1) / original_x1
    # MAPE= np.mean(APE)
    if APE.isnull().any():
        return 1
    else:
        return np.sqrt(np.mean(APE**2))+ L2#均方根误差和作为损失函数

In [3]:
def target():
    for i in range(10):
        pop = random.randint(1,200)
        max_iter = random.randint(1,1500)

        # print(f'{cnt} 的 参数为 pop: {pop}, max_iter: {max_iter}')

        # 调用粒子群，指定上述五个变量的取值范围，设置粒子群参数，最终得到最优的 zeta, tau, gamma
        pso = PSO(func=func, n_dim=3, pop=pop, max_iter=max_iter, lb=[0, -10, 0], ub=[1, 10, 10])
        pso.record_mode = True
        pso.run()


        result.append((pop, max_iter, pso.gbest_x[0], pso.gbest_x[1], pso.gbest_x[2], pso.gbest_y[0]))

        # print(cnt, end=' ')
        print((pop, max_iter, pso.gbest_x[0], pso.gbest_x[1], pso.gbest_x[2], pso.gbest_y[0]))

In [4]:
result = []

data = pd.read_excel("京津冀3省2014-2023使用版数据-2位小数.xlsx")
data = data.iloc[:, :]
n = data.shape[0]
train_n = n-2

In [ ]:
# 创建多个线程
threads = [threading.Thread(target=target) for _ in range(20)]

# 启动线程
for thread in threads:
    thread.start()

# 等待所有线程完成
for thread in threads:
    thread.join()

D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in power
D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


(44, 8, 0.586636105935045, 0.4365540862491193, 0.0, 0.22832318932578097)


D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:82: RuntimeWarning: overflow encountered in power


(6, 1449, 0.8846038591194199, -0.005801036859195122, 1.3756465525323684, 0.08374911824318414)
(21, 512, 0.7597252978770498, 0.008721239007296058, 1.880208832909174, 0.10294169659965434)


D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars


(8, 652, 0.015222065764806855, 0.39020041532080935, 5.596906454172672, 0.10886186765767063)
(31, 636, 2.705169956356516e-12, 0.1637833945691811, 7.861726701646864, 0.06669104674578484)
(114, 184, 2.5666247988506056e-12, 0.1637740714592352, 7.862045294645994, 0.06669505304439682)
(29, 704, 1.0, -0.02769417665816855, 1.1433480561122236, 0.0716598588262537)
(22, 1039, 1.0, -0.0276939073820007, 1.1433491182142996, 0.07165985882116797)
(24, 982, 0.2931054677405604, -0.8312779353688989, 9.208860093142917, 0.12036338649775193)
(13, 678, 3.3689314205032134e-07, 0.17267832774592493, 7.787257812804489, 0.0672674395709736)
(110, 320, 0.3076252252443335, -0.48885475733900613, 5.922318678940024, 0.11920181428617758)
(86, 405, 0.24907531968149435, 0.098919852091425, 1.3870625864097783, 0.0984583329727266)
(124, 36, 1.0, 0.341707067150095, 0.0, 0.12692230393474344)
(195, 224, 1.9431505205812284e-12, 0.16342244177521922, 7.862016550262453, 0.06665711805259598)
(67, 134, 1.0, 0.3415821161708056, 0.0, 0

In [ ]:
# with ThreadPoolExecutor(max_workers=100) as executor:
#     futures = []
#     for i in range(100):
#         # 提交任务到线程池
#         future = executor.submit(target, i)
#         futures.append(future)
#
#     # 等待所有线程（任务）完成
#     for future in futures:
#         print("线程数量： ", executor.work_count())
#         future.result()

In [ ]:
# threads = []
# for i in range(1000):
#     t = threading.Thread(target=target)
#     t.start()
#     threads.append(t)
#
# # 等待所有线程完成
# for t in threads:
#     t.join()
#
# pd.DataFrame(result).to_excel('file/01.xlsx')

In [ ]:
pd.DataFrame(result).to_excel('case2.xlsx')